In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np
from collections import defaultdict

# Constants
MAX_SENT_LENGTH = 30
MAX_SENTS = 50
EMBEDDING_DIM = 300
NUM_HEADS = 20
HEAD_SIZE = 20
DROPOUT_RATE = 0.2
NPRATIO = 4  # Number of negative samples per positive
BATCH_SIZE = 64
EPOCHS = 1

In [11]:
# Load Datasets
articles = pd.read_csv("Data/articles.csv")
behaviors_train = pd.read_csv("Data/behaviors_train.csv")
behaviors_val = pd.read_csv("Data/behaviors_val.csv")
history_train = pd.read_csv("Data/history_train.csv")
history_val = pd.read_csv("Data/history_val.csv")

In [12]:
# Step 1: Tokenize article titles and build vocabulary
def build_vocab_and_tokenize(titles, max_len=MAX_SENT_LENGTH):
    vocab = defaultdict(lambda: len(vocab))  # Default dictionary for token ids
    vocab["<PAD>"] = 0  # Padding token
    vocab["<UNK>"] = 1  # Unknown token

    VOCAB_SIZE = len(vocab)

    tokenized_titles = []
    for title in titles:
        tokens = title.lower().split()[:max_len]  # Simple whitespace tokenizer, truncate to max_len
        tokenized = [vocab[token] for token in tokens]
        tokenized = [min(idx, VOCAB_SIZE - 1) for idx in tokenized]  # Ensure indices are valid
        padded = tokenized + [vocab["<PAD>"]] * (max_len - len(tokenized))  # Padding to max_len
        tokenized_titles.append(padded)

    return tokenized_titles, vocab, VOCAB_SIZE

articles["tokenized_title"], vocab, VOCAB_SIZE = build_vocab_and_tokenize(articles["title"].fillna("<UNK>"))

In [13]:
# Step 2: Clean and process user history
def clean_article_ids(article_ids):
    try:
        if "..." in article_ids:
            return None  # Mark for removal
        return list(map(int, article_ids.strip("[]").split()))
    except ValueError:
        return None

history_train["cleaned_article_ids"] = history_train["article_id_fixed"].apply(clean_article_ids)
history_val["cleaned_article_ids"] = history_val["article_id_fixed"].apply(clean_article_ids)

history_train_cleaned = history_train.dropna(subset=["cleaned_article_ids"]).reset_index(drop=True)
history_val_cleaned = history_val.dropna(subset=["cleaned_article_ids"]).reset_index(drop=True)

def process_cleaned_user_history(cleaned_history_df):
    user_histories = defaultdict(list)
    for _, row in cleaned_history_df.iterrows():
        user_id = row["user_id"]
        article_ids = row["cleaned_article_ids"]
        user_histories[user_id].extend(article_ids)
    return user_histories

user_history_train_cleaned = process_cleaned_user_history(history_train_cleaned)
user_history_val_cleaned = process_cleaned_user_history(history_val_cleaned)

In [14]:
# Step 3: Create samples and labels
def create_samples(behaviors_df, user_history, max_sents=MAX_SENTS):
    samples = []
    labels = []
    for _, row in behaviors_df.iterrows():
        user_id = row["user_id"]
        clicked_articles = list(map(int, row["article_ids_clicked"].strip("[]").split()))
        inview_articles = list(map(int, row["article_ids_inview"].strip("[]").split()))

        user_hist = user_history[user_id][:max_sents] + [0] * (max_sents - len(user_history[user_id]))
        user_hist = [min(idx, VOCAB_SIZE - 1) for idx in user_hist]  # Ensure valid indices

        for article_id in inview_articles:
            article_id = min(article_id, VOCAB_SIZE - 1)  # Ensure valid index
            label = 1 if article_id in clicked_articles else 0
            samples.append((user_hist, article_id))
            labels.append(label)

    return samples, labels

train_samples_cleaned, train_labels_cleaned = create_samples(behaviors_train, user_history_train_cleaned)
val_samples_cleaned, val_labels_cleaned = create_samples(behaviors_val, user_history_val_cleaned)

In [15]:

# Step 4: Define PyTorch Dataset
class NewsDataset(Dataset):
    def __init__(self, samples, labels):
        self.samples = samples
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        user_history, candidate = self.samples[idx]
        user_history_padded = [0] * MAX_SENTS  # Pad user history
        user_history_padded[:len(user_history)] = user_history[:MAX_SENTS]
        candidate_padded = [candidate] + [0] * (MAX_SENT_LENGTH - 1)  # Pad candidate to sequence
        return (
            torch.tensor(user_history_padded, dtype=torch.long),
            torch.tensor(candidate_padded, dtype=torch.long),
            torch.tensor(self.labels[idx], dtype=torch.long),
        )

train_dataset = NewsDataset(train_samples_cleaned, train_labels_cleaned)
val_dataset = NewsDataset(val_samples_cleaned, val_labels_cleaned)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [16]:
# Step 5: Define the NRMS Model
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.num_heads = num_heads
        self.head_size = head_size
        self.output_dim = num_heads * head_size
        self.qkv_linear = nn.Linear(EMBEDDING_DIM, self.output_dim * 3)
        self.fc_out = nn.Linear(self.output_dim, EMBEDDING_DIM)

    def forward(self, x):
        if len(x.size()) != 3:
            raise ValueError(f"Expected input to be 3D (batch_size, seq_length, embed_dim), got {x.size()}")
        batch_size, seq_length, embed_dim = x.size()
        qkv = self.qkv_linear(x).reshape(batch_size, seq_length, self.num_heads, 3 * self.head_size)
        qkv = qkv.permute(2, 0, 1, 3)
        Q, K, V = torch.chunk(qkv, 3, dim=-1)
        scores = torch.matmul(Q, K.transpose(-2, -1)) / np.sqrt(self.head_size)
        attention = F.softmax(scores, dim=-1)
        weighted = torch.matmul(attention, V)
        return self.fc_out(weighted.permute(1, 2, 0, 3).reshape(batch_size, seq_length, self.output_dim))

class TitleEncoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.dropout = nn.Dropout(DROPOUT_RATE)
        self.self_attention = MultiHeadSelfAttention(NUM_HEADS, HEAD_SIZE)
        self.dense = nn.Linear(embedding_dim, 1)

    def forward(self, x):
        x = self.embedding(x)  # Ensure input is embedded
        if len(x.size()) != 3:
            raise ValueError(f"Embedding layer output should be 3D, got {x.size()}")
        x = self.dropout(x)
        x = self.self_attention(x)
        attention_weights = F.softmax(self.dense(x).squeeze(-1), dim=-1)
        return torch.sum(x * attention_weights.unsqueeze(-1), dim=1)

class NRMS(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_classes):
        super().__init__()
        self.title_encoder = TitleEncoder(vocab_size, embedding_dim)

    def forward(self, candidates, user_history):
        user_rep = self.title_encoder(user_history)  # Output: (batch_size, embedding_dim)
        candidate_rep = self.title_encoder(candidates)  # Output: (batch_size, embedding_dim)
        return torch.matmul(candidate_rep, user_rep.unsqueeze(-1)).squeeze(-1)

In [17]:
# Step 6: Train the Model
model = NRMS(VOCAB_SIZE, EMBEDDING_DIM, NPRATIO + 1)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

def train_model(model, train_loader, val_loader, optimizer, criterion, epochs=EPOCHS):
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for batch in train_loader:
            user_histories, candidates, labels = batch
            optimizer.zero_grad()
            outputs = model(candidates, user_histories)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch + 1}, Loss: {total_loss:.4f}")

In [18]:
# Train the model
train_model(model, train_loader, val_loader, optimizer, criterion)

Epoch 1, Loss: 20809.6994


In [22]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate_model(model, data_loader):
    model.eval()  # Set model to evaluation mode
    all_labels = []
    all_predictions = []
    all_probabilities = []

    with torch.no_grad():  # Disable gradient computation
        for batch in data_loader:
            user_histories, candidates, labels = batch
            
            # Get raw logits from the model
            outputs = model(candidates, user_histories)
            
            # Compute probabilities for the positive class
            probabilities = torch.softmax(outputs, dim=1)[:, 1]
            
            # Get the predicted class
            predictions = torch.argmax(outputs, dim=1)

            # Store labels, predictions, and probabilities
            all_labels.extend(labels.numpy())
            all_predictions.extend(predictions.numpy())
            all_probabilities.extend(probabilities.numpy())

    # Calculate evaluation metrics
    accuracy = accuracy_score(all_labels, all_predictions)
    precision = precision_score(all_labels, all_predictions, average='weighted')
    recall = recall_score(all_labels, all_predictions, average='weighted')
    f1 = f1_score(all_labels, all_predictions, average='weighted')

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

    return accuracy, precision, recall, f1

# Evaluate on validation data
accuracy, precision, recall, f1 = evaluate_model(model, val_loader)

Accuracy: 0.8732
Precision: 1.0000
Recall: 0.8732
F1 Score: 0.9323


/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
